# Pretrained  GPT2  Model Deployment Example

In this notebook, we will run an example of text generation using GPT2 model exported from HuggingFace and deployed with Seldon's Triton pre-packed server. the example also covers converting the model to ONNX format.
The implemented example below is of the Greedy approach for the next token prediction.
more info: https://huggingface.co/transformers/model_doc/gpt2.html?highlight=gpt2

After we have the module deployed to Kubernetes, we will run a simple load test to evaluate the module inference performance.


## Steps:
1. Download pretrained GPT2 model from hugging face
2. Convert the model to ONNX
3. Store it in MinIo bucket
4. Setup Seldon-Core in your kubernetes cluster
5. Deploy the ONNX model with Seldon’s prepackaged Triton server.
6. Interact with the model, run a greedy alg example (generate sentence completion)
7. Run load test using vegeta
8. Clean-up

## Basic requirements
* Helm v3.0.0+
* A Kubernetes cluster running v1.13 or above (minkube / docker-for-windows work well if enough RAM)
* kubectl v1.14+
* Python 3.6+ 

In [ ]:
%%writefile requirements.txt
transformers==4.5.1
torch==1.8.1
tokenizers<0.11,>=0.10.1
tensorflow==2.4.1
tf2onnx

In [ ]:
!pip install --trusted-host=pypi.python.org --trusted-host=pypi.org --trusted-host=files.pythonhosted.org -r requirements.txt


### Export HuggingFace TFGPT2LMHeadModel pre-trained model and save it locally

In [ ]:
from transformers import TFGPT2LMHeadModel, GPT2Tokenizer
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
model = TFGPT2LMHeadModel.from_pretrained("gpt2", from_pt=True, pad_token_id=tokenizer.eos_token_id)
model.save_pretrained("./tfgpt2model", saved_model=True)

### Convert the TensorFlow saved model to ONNX

In [ ]:
!python -m tf2onnx.convert --saved-model ./tfgpt2model/saved_model/1 --opset 11  --output model.onnx

## Azure Setup
We  have provided [Azure Setup Notebook](AzureSetup.ipynb) that deploys AKS cluster, Azure storage account and installs Azure Blob CSI driver. If AKS cluster already exists skip to creation of Blob Storage and CSI driver installtion steps.

In [4]:
resource_group = "seldon"   # feel free to replace or use this default
aks_name = "modeltests"    

storage_account_name = "modeltestsgpt"        # fill in
storage_container_name = "gpt2onnx"             

### Copy your model to Azure Blob


In [ ]:
%%time
# Copy model file
!az extension add --name storage-preview
!az storage azcopy blob upload --container {storage_container_name} \
                               --account-name {storage_account_name} \
                               --source  ./model.onnx \
                               --destination gpt2/1/model.onnx  

In [3]:
#Verify Uploaded file
!az storage blob list \
    --account-name {storage_account_name}\
    --container-name {storage_container_name} \
    --output table 
    

This command has been deprecated and will be removed in future release. Use 'az storage fs file list' instead. For more information go to https://github.com/Azure/azure-cli/blob/dev/src/azure-cli/azure/cli/command_modules/storage/docs/ADLS%20Gen2.md
The behavior of this command has been altered by the following extension: storage-preview
Name               IsDirectory    Blob Type    Blob Tier    Length     Content Type              Last Modified              Snapshot
-----------------  -------------  -----------  -----------  ---------  ------------------------  -------------------------  ----------
gpt2/1/model.onnx                 BlockBlob    Hot          652535462  application/octet-stream  2021-05-28T04:37:11+00:00


##  Add Azure PersistentVolume and Claim 

In [ ]:
key = !az storage account keys list --account-name {storage_account_name} -g {resource_group} --query '[0].value' -o tsv
storage_account_key = key[0] 

For more details on creating PersistentVolume using CSI driver refer to https://github.com/kubernetes-sigs/blob-csi-driver/blob/master/deploy/example/e2e_usage.md

In [ ]:
# Create secret to access storage account
!kubectl create secret generic azure-blobsecret --from-literal azurestorageaccountname={storage_account_name} --from-literal azurestorageaccountkey="{storage_account_key}" --type=Opaque 

In [ ]:
%%writefile azure-blobfuse-pv.yaml
apiVersion: v1
kind: PersistentVolume
metadata:
  name: pv-gptblob
  
spec:
  capacity:
    storage: 10Gi
  accessModes:
    - ReadWriteMany
  persistentVolumeReclaimPolicy: Retain  # "Delete" is not supported in static provisioning
  csi:
    driver: blob.csi.azure.com
    readOnly: false
    volumeHandle: trainingdata  # make sure this volumeid is unique in the cluster
    volumeAttributes:
      containerName: gpt2onnx # Modify if changed in Notebook
    nodeStageSecretRef:
      name: azure-blobsecret
      namespace: default
    
---
kind: PersistentVolumeClaim
apiVersion: v1
metadata:
  name: pvc-gptblob
 
spec:
  accessModes:
    - ReadWriteMany
  resources:
    requests:
      storage: 10Gi
  volumeName: pv-gptblob
  storageClassName: ""


In [ ]:
!kubectl apply -f  azure-blobfuse-pv.yaml

In [11]:
# Verify PVC is bound
!kubectl get pv,pvc 

NAME                          CAPACITY   ACCESS MODES   RECLAIM POLICY   STATUS        CLAIM                 STORAGECLASS   REASON   AGE
persistentvolume/pv-blob      10Gi       RWX            Retain           Terminating   default/pvc-blob                              127m
persistentvolume/pv-gptblob   10Gi       RWX            Retain           Bound         default/pvc-gptblob                           15m

NAME                                STATUS        VOLUME       CAPACITY   ACCESS MODES   STORAGECLASS   AGE
persistentvolumeclaim/pvc-blob      Terminating   pv-blob      10Gi       RWX                           127m
persistentvolumeclaim/pvc-gptblob   Bound         pv-gptblob   10Gi       RWX                           15m


### Run Seldon in your kubernetes cluster

Follow the [Seldon-Core Setup notebook](https://docs.seldon.io/projects/seldon-core/en/latest/examples/seldon_core_setup.html) to Setup a cluster with Ambassador Ingress or Istio and install Seldon Core

### Deploy your model with Seldon pre-packaged Triton server

In [8]:
%%writefile gpt2-deploy.yaml
apiVersion: machinelearning.seldon.io/v1alpha2
kind: SeldonDeployment
metadata:
  name: gpt2
spec:
  predictors:
  - graph:
      implementation: TRITON_SERVER
      logger:
        mode: all
      modelUri: pvc://pvc-gptblob
      name: gpt2
      type: MODEL
    name: default
    replicas: 1
  protocol: kfserving

Overwriting gpt2-deploy.yaml


In [9]:

!kubectl apply -f gpt2-deploy.yaml -n default

seldondeployment.machinelearning.seldon.io/gpt2 created


In [10]:
!kubectl rollout status deploy/$(kubectl get deploy -l seldon-deployment-id=gpt2 -o jsonpath='{.items[0].metadata.name}')

Waiting for deployment "gpt2-default-0-gpt2" rollout to finish: 0 of 1 updated replicas are available...
error: deployment "gpt2-default-0-gpt2" exceeded its progress deadline


#### Interact with the model: get model metadata (a "test" request to make sure our model is available and loaded correctly)

In [ ]:
ingress_ip=!(kubectl get svc --namespace seldon-system ambassador -o jsonpath='{.status.loadBalancer.ingress[0].ip}')
ingress_ip = ingress_ip[0]

!curl -v http://{ingress_ip}:80/seldon/default/gpt2/v2/models/gpt2

### Run prediction test: generate a sentence completion using GPT2 model  - Greedy approach


In [ ]:
import requests
import json
import numpy as np
from transformers import GPT2Tokenizer

tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
input_text = 'I enjoy working in Seldon'
count = 0
max_gen_len = 10
gen_sentence = input_text
while count < max_gen_len:
    input_ids = tokenizer.encode(gen_sentence, return_tensors='tf')
    shape = input_ids.shape.as_list()
    payload = {
            "inputs": [
                {"name": "input_ids:0",
                 "datatype": "INT32",
                 "shape": shape,
                 "data": input_ids.numpy().tolist()
                 },
                {"name": "attention_mask:0",
                 "datatype": "INT32",
                 "shape": shape,
                 "data": np.ones(shape, dtype=np.int32).tolist()
                 }
                ]
            }

    ret = requests.post('http://localhost:80/seldon/default/gpt2/v2/models/gpt2/infer', json=payload)

    try:
        res = ret.json()
    except:
       continue

    # extract logits
    logits = np.array(res["outputs"][1]["data"])
    logits = logits.reshape(res["outputs"][1]["shape"])

    # take the best next token probability of the last token of input ( greedy approach)
    next_token = logits.argmax(axis=2)[0]
    next_token_str = tokenizer.decode(next_token[-1:], skip_special_tokens=True,
                                      clean_up_tokenization_spaces=True).strip()
    gen_sentence += ' ' + next_token_str
    count += 1

print(f'Input: {input_text}\nOutput: {gen_sentence}')

### Run Load Test / Performance Test using vegeta

#### Install vegeta, for more details take a look in [vegeta](https://github.com/tsenart/vegeta#install) official documentation

In [ ]:
!wget https://github.com/tsenart/vegeta/releases/download/v12.8.3/vegeta-12.8.3-linux-amd64.tar.gz
!tar -zxvf vegeta-12.8.3-linux-amd64.tar.gz
!chmod +x vegeta

#### Generate vegeta [target file](https://github.com/tsenart/vegeta#-targets) contains "post" cmd with payload in the requiered structure

In [ ]:
from subprocess import run, Popen, PIPE
import json
import numpy as np
from transformers import TFGPT2LMHeadModel, GPT2Tokenizer
import base64

tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
input_text = 'I enjoy working in Seldon'
input_ids = tokenizer.encode(input_text, return_tensors='tf')
shape = input_ids.shape.as_list()
payload = {
		"inputs": [
			{"name": "input_ids:0",
			 "datatype": "INT32",
			 "shape": shape,
			 "data": input_ids.numpy().tolist()
			 },
			{"name": "attention_mask:0",
			 "datatype": "INT32",
			 "shape": shape,
			 "data": np.ones(shape, dtype=np.int32).tolist()
			 }
			]
		}

cmd= {"method": "POST",
		"header": {"Content-Type": ["application/json"] },
		"url": "http://localhost:80/seldon/default/gpt2/v2/models/gpt2/infer",
		"body": base64.b64encode(bytes(json.dumps(payload), "utf-8")).decode("utf-8")}

with open("vegeta_target.json", mode="w") as file:
	json.dump(cmd, file)
	file.write('\n\n')

In [ ]:
!vegeta attack -targets=vegeta_target.json -rate=1 -duration=60s -format=json | vegeta report -type=text

### Clean-up

In [ ]:
!kubectl delete -f gpt2-deploy.yaml -n default